## 4a)

In [1]:
#Start Spark RDD Session
import re
import sys
from pyspark import SparkConf, SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 09:56:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/07 09:56:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
#Bullet Point 1
#Find total number of user/reviewer rating records
mov_rating = sc.textFile("mov_rating.dat")
mov_rating.collect()

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013',
 '62\t257\t2\t879372434',
 '286\t1014\t5\t879781125',
 '200\t222\t5\t876042340',
 '210\t40\t3\t891035994',
 '224\t29\t3\t888104457',
 '303\t785\t3\t879485318',
 '122\t387\t5\t879270459',
 '194\t274\t2\t879539794',
 '291\t1042\t4\t874834944',
 '234\t1184\t2\t892079237',
 '119\t392\t4\t886176814',
 '167\t486\t4\t892738452',
 '299\t144\t4\t877881320',
 '291\t118\t2\t874833878',
 '308\t1\t4\t887736532',
 '95\t546\t2\t879196566',
 '38\t95\t5\t892430094',
 '102\t768\t2\t883748450',
 '63\t277\t4\t875747401',
 '160\t234\t5\t876861185',
 '50\t246\t3\t877052329',
 '301\t98\t4\t882075827',
 '225\t193\t4\t879539727',
 '290\t88\t4\t880731963',
 '97\t194\t3\t884238860',
 '157\t274\t4\t886890835',
 '181\t1081\t1\t878962623',
 '278\t603\t5\t

In [7]:
mov_rating.count()

100000

In [8]:
#Bullet Point 2
#Find unique number of reviewers
reviewers = mov_rating.map(lambda line: line.split('\t')[0]).distinct()
reviewers.count()

943

In [9]:
#Bullet Point 3
#Find the unique number of movies reviewed
movies_reviewed = mov_rating.map(lambda line: line.split('\t')[1]).distinct()
movies_reviewed.count()

1682

In [10]:
#Bullet Point 4
#Find top 10 reviewers. Show reviewer ID and total reviewed movie count. Sort descending order.

#get the reviwer's user id and the movies that the user has reviewed
reviewer = mov_rating.map(lambda line: line.split('\t')[0])
movies = mov_rating.map(lambda line: line.split('\t')[1])

#Zip the two variables together
movie_reviews = reviewer.zip(movies)

#group the rdd by the key which in this case is the reviewer and map the movie id's reviews to a list
#get the length of the list which signifies the total number of movies reviwed and sort number of movies 
#reviewed in descending order
#take top 10 records
movie_reviews = movie_reviews.groupByKey().mapValues(list).map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=False)
movie_reviews.take(10)

[('405', 737),
 ('655', 685),
 ('13', 636),
 ('450', 540),
 ('276', 518),
 ('416', 493),
 ('537', 490),
 ('303', 484),
 ('234', 480),
 ('393', 448)]

In [11]:
#Bullet Point 5
#Find top 10 popular movies. Show movie ID, movie name, reviewed counts. Sort descending order.

#load the mov_item file
mov_item = sc.textFile("mov_item.dat")
#get the movie id and movie name
movie_id = mov_item.map(lambda line: line.split('|')[0])
movie_name = mov_item.map(lambda line: line.split('|')[1])

#Zip the two variables together to form the movie collection
movie_collection = movie_id.zip(movie_name)

#Zip the movies and reviewer RDDs from the previous bullet point and group the rdd by the key which 
#in this case is the movie ID and map the movie id's reviews to a list
#get the length of the list which signifies the total number of movies reviwed and sort number of movies 
#reviewed in descending order
popular_movies = movies.zip(reviewer).groupByKey().mapValues(list).map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=False)

#Join the list of movies together with the popular movie collection
top_movies = movie_collection.join(popular_movies)

#Extract the top 10 popular movies
top_movies.take(10)

[('4', ('Get Shorty (1995)', 209)),
 ('5', ('Copycat (1995)', 86)),
 ('10', ('Richard III (1995)', 89)),
 ('16', ('French Twist (Gazon maudit) (1995)', 39)),
 ('17', ('From Dusk Till Dawn (1996)', 92)),
 ('21', ('Muppet Treasure Island (1996)', 84)),
 ('22', ('Braveheart (1995)', 297)),
 ('26', ('Brothers McMullen, The (1995)', 73)),
 ('29', ('Batman Forever (1995)', 114)),
 ('32', ('Crumb (1994)', 81))]

## 4b

In [12]:
#Bullet Point 1
#Find the total number of movies released each year

#from the mov_item file, get the second column which is the release date and extract out the 
#last 4 characters of the date which is the year and count the number of movies for each year 
#and sort based on ascending year
#filter out the year that is empty with 1 movie released
year_released = sorted(mov_item.map(lambda line: line.split('|')[2][-4:]).filter(lambda line: line != '').countByValue().items())

#print the output
for row in year_released:
    print(f'Year: {row[0]}, Number of Movies Released: {row[1]}')

Year: 1922, Number of Movies Released: 1
Year: 1926, Number of Movies Released: 1
Year: 1930, Number of Movies Released: 1
Year: 1931, Number of Movies Released: 1
Year: 1932, Number of Movies Released: 1
Year: 1933, Number of Movies Released: 2
Year: 1934, Number of Movies Released: 4
Year: 1935, Number of Movies Released: 4
Year: 1936, Number of Movies Released: 2
Year: 1937, Number of Movies Released: 4
Year: 1938, Number of Movies Released: 3
Year: 1939, Number of Movies Released: 7
Year: 1940, Number of Movies Released: 8
Year: 1941, Number of Movies Released: 5
Year: 1942, Number of Movies Released: 2
Year: 1943, Number of Movies Released: 4
Year: 1944, Number of Movies Released: 5
Year: 1945, Number of Movies Released: 4
Year: 1946, Number of Movies Released: 5
Year: 1947, Number of Movies Released: 5
Year: 1948, Number of Movies Released: 3
Year: 1949, Number of Movies Released: 4
Year: 1950, Number of Movies Released: 7
Year: 1951, Number of Movies Released: 5
Year: 1952, Numb

In [13]:
#Bullet Point 2
#Find the range of movies released year
release_year = mov_item.map(lambda line: line.split('|')[2][-4:]).filter(lambda line: line != '')
print(f'The range of movies released year is from {release_year.min()} to {release_year.max()} ')

The range of movies released year is from 1922 to 1998 


In [14]:
#Bullet Point 3
#for the year with highest count, take top 3 most reviewed movies of each genre
#sort by average review ratings descending
#show genre name, movieID, movie name, average rating
#save result using RDD file saving mechanism and show content

In [15]:
#The steps below are for the third bullet point

In [16]:
#Step 1: Display the year that has the highest number of movies released.

#from the year_released rdd above, based on the count, get the max movie count together with the year
highest_count = max(mov_item.map(lambda line: line.split('|')[2][-4:]).filter(lambda line: line != '').countByValue().items(), key=lambda x: x[1])
#store the year which is the first index of the (year,count)
year_max = highest_count[0]
#print the year
print(year_max)

1996


In [17]:
#Step 2: Filter out the movies from the year 1996
highest_count = max(mov_item.map(lambda line: line.split('|')[2][-4:]).filter(lambda line: line != '').countByValue().items(), key=lambda x: x[1])
year_max = highest_count[0]

#based on the two lines of code that gives us the year, the mov_item rdd is filtered on the release date
#column and prints the movies that are released in 1996
busyyear_movies = mov_item.filter(lambda line: year_max in line.split('|')[2])
busyyear_movies.collect()

['10|Richard III (1995)|22-Jan-1996||http://us.imdb.com/M/title-exact?Richard%20III%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|1|0',
 "15|Mr. Holland's Opus (1995)|29-Jan-1996||http://us.imdb.com/M/title-exact?Mr.%20Holland's%20Opus%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0",
 '17|From Dusk Till Dawn (1996)|05-Feb-1996||http://us.imdb.com/M/title-exact?From%20Dusk%20Till%20Dawn%20(1996)|0|1|0|0|0|1|1|0|0|0|0|1|0|0|0|0|1|0|0',
 '21|Muppet Treasure Island (1996)|16-Feb-1996||http://us.imdb.com/M/title-exact?Muppet%20Treasure%20Island%20(1996)|0|1|1|0|0|1|0|0|0|0|0|0|1|0|0|0|1|0|0',
 '22|Braveheart (1995)|16-Feb-1996||http://us.imdb.com/M/title-exact?Braveheart%20(1995)|0|1|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|1|0',
 '23|Taxi Driver (1976)|16-Feb-1996||http://us.imdb.com/M/title-exact?Taxi%20Driver%20(1976)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|1|0|0',
 '24|Rumble in the Bronx (1995)|23-Feb-1996||http://us.imdb.com/M/title-exact?Hong%20Faan%20Kui%20(1995)|0|1|1|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0',
 

In [18]:
#Step 3: Map the genres to each of the movie

#from the rdd above, we only need the Movie ID, Movie name and the genre.
#Filter out the valid fields and place the binary values of genre in a list for easier viewing
valid_fields = busyyear_movies.map(lambda line: line.split('|')).map(lambda columns: (int(columns[0]), columns[1], [int(x) for x in columns[5:]]))

#collect the genre descriptions in a list
genre_description = sc.textFile("mov_genre.dat").map(lambda line: line.split('|')[0]).collect()

#create a function to map the genre to the binary values
#if the value is 1 in the list, that index is to be updated based on the same index in the genre_description
#the mapped genres are returned
def genre_mapping(genre):
    mapped_genres = [genre_description[i] for i, value in enumerate(genre) if value == 1]
    return mapped_genres

#The function is then used to map each genre in the list for each movie
mapped_movies = valid_fields.map(lambda x: (x[0], x[1], genre_mapping(x[2])))

mapped_movies.collect()

[(10, 'Richard III (1995)', ['Drama', 'War']),
 (15, "Mr. Holland's Opus (1995)", ['Drama']),
 (17,
  'From Dusk Till Dawn (1996)',
  ['Action', 'Comedy', 'Crime', 'Horror', 'Thriller']),
 (21,
  'Muppet Treasure Island (1996)',
  ['Action', 'Adventure', 'Comedy', 'Musical', 'Thriller']),
 (22, 'Braveheart (1995)', ['Action', 'Drama', 'War']),
 (23, 'Taxi Driver (1976)', ['Drama', 'Thriller']),
 (24, 'Rumble in the Bronx (1995)', ['Action', 'Adventure', 'Crime']),
 (25, 'Birdcage, The (1996)', ['Comedy']),
 (93, 'Welcome to the Dollhouse (1995)', ['Comedy', 'Drama']),
 (103,
  'All Dogs Go to Heaven 2 (1996)',
  ['Animation', "Children's", 'Musical']),
 (104, 'Theodore Rex (1995)', ['Comedy']),
 (105, 'Sgt. Bilko (1996)', ['Comedy']),
 (106, 'Diabolique (1996)', ['Drama', 'Thriller']),
 (107, 'Moll Flanders (1996)', ['Drama']),
 (108, 'Kids in the Hall: Brain Candy (1996)', ['Comedy']),
 (109, 'Mystery Science Theater 3000: The Movie (1996)', ['Comedy', 'Sci-Fi']),
 (111, 'Truth About 

In [19]:
#Step 4: Get the respective ratings for each movie in the Rdd above

#get the list of movie_ids
titles_released = mapped_movies.map(lambda x: x[0]).collect()
#load the mov_rating file and view lines with the movie_id collected from titles_released
mov_rating = sc.textFile("mov_rating.dat").map(lambda line: line.split('\t')).map(lambda columns: (int(columns[0]), int(columns[1]), int(columns[2]), columns[3])).filter(lambda x: (x[1] in titles_released))
#for each movie_id, get the respective list of ratings
mov_rating = mov_rating.map(lambda x: (x[1], x[2])).groupByKey().mapValues(list)
#get the movie_id, number of ratings for each movie and average rating for each movie and set movie_id as key
final_filtered_ratings = mov_rating.map(lambda line: (line[0], len(line[1]), sum(int(rating) for rating in line[1]) / len(line[1]))).map(lambda x: (x[0], (x[1], x[2])))
final_filtered_ratings.collect()

[(222, (365, 3.66027397260274)),
 (118, (293, 3.2150170648464163)),
 (546, (254, 3.031496062992126)),
 (304, (149, 3.5369127516778525)),
 (288, (478, 3.4414225941422596)),
 (832, (22, 2.5454545454545454)),
 (368, (31, 1.903225806451613)),
 (1134, (38, 3.3157894736842106)),
 (1336, (10, 1.8)),
 (1240, (26, 3.6538461538461537)),
 (930, (80, 2.7)),
 (476, (160, 3.01875)),
 (220, (66, 3.3636363636363638)),
 (280, (85, 3.1882352941176473)),
 (408, (112, 4.491071428571429)),
 (1098, (42, 3.6904761904761907)),
 (10, (89, 3.831460674157303)),
 (106, (71, 2.887323943661972)),
 (146, (10, 2.5)),
 (508, (215, 3.5953488372093023)),
 (410, (162, 3.1296296296296298)),
 (1086, (21, 3.2857142857142856)),
 (286, (481, 3.656964656964657)),
 (758, (21, 1.7142857142857142)),
 (596, (127, 3.377952755905512)),
 (1060, (39, 3.051282051282051)),
 (278, (60, 3.2333333333333334)),
 (742, (267, 3.644194756554307)),
 (762, (115, 3.391304347826087)),
 (406, (49, 2.4489795918367347)),
 (150, (157, 3.828025477707006

In [20]:
#Step 5: Rearrange the mapped_movies RDD to get the genre of each movie as the key
genre_movie_pairs = mapped_movies.flatMap(lambda x: [(genre, (x[0], x[1])) for genre in x[2]])
genre_movie_pairs.collect()

[('Drama', (10, 'Richard III (1995)')),
 ('War', (10, 'Richard III (1995)')),
 ('Drama', (15, "Mr. Holland's Opus (1995)")),
 ('Action', (17, 'From Dusk Till Dawn (1996)')),
 ('Comedy', (17, 'From Dusk Till Dawn (1996)')),
 ('Crime', (17, 'From Dusk Till Dawn (1996)')),
 ('Horror', (17, 'From Dusk Till Dawn (1996)')),
 ('Thriller', (17, 'From Dusk Till Dawn (1996)')),
 ('Action', (21, 'Muppet Treasure Island (1996)')),
 ('Adventure', (21, 'Muppet Treasure Island (1996)')),
 ('Comedy', (21, 'Muppet Treasure Island (1996)')),
 ('Musical', (21, 'Muppet Treasure Island (1996)')),
 ('Thriller', (21, 'Muppet Treasure Island (1996)')),
 ('Action', (22, 'Braveheart (1995)')),
 ('Drama', (22, 'Braveheart (1995)')),
 ('War', (22, 'Braveheart (1995)')),
 ('Drama', (23, 'Taxi Driver (1976)')),
 ('Thriller', (23, 'Taxi Driver (1976)')),
 ('Action', (24, 'Rumble in the Bronx (1995)')),
 ('Adventure', (24, 'Rumble in the Bronx (1995)')),
 ('Crime', (24, 'Rumble in the Bronx (1995)')),
 ('Comedy', (25

In [21]:
#Step 6: Create new RDD to join the ratings rdd and the genre_movie_pairs rdd based on movie_id 
#to get an RDD with genre, movie_id, movie_name and corresponding average rating

#Initialize an empty list
result = []
#Create a for loop for both ratings and genre_movie rdd and append 
#genre, movie_id, movie_name and corresponding average rating only if the movie_id between the two RDDs
#are the same
for genre, (movie_id, movie_name) in genre_movie_pairs.collect():
    for movie_id2, (total_reviews, avg_rating) in final_filtered_ratings.collect():
        if movie_id == movie_id2:
            result.append((genre, (movie_id, movie_name, avg_rating)))
            
#paralleize the list into an RDD and display the result
result = sc.parallelize(result)
result.collect()

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=19747Kb max_used=19747Kb free=111324Kb
 bounds [0x0000000102e3c000, 0x00000001041ac000, 0x000000010ae3c000]
 total_blobs=8296 nmethods=7430 adapters=780
 compilation: disabled (not enough contiguous free space left)


[('Drama', (10, 'Richard III (1995)', 3.831460674157303)),
 ('War', (10, 'Richard III (1995)', 3.831460674157303)),
 ('Drama', (15, "Mr. Holland's Opus (1995)", 3.7781569965870307)),
 ('Action', (17, 'From Dusk Till Dawn (1996)', 3.119565217391304)),
 ('Comedy', (17, 'From Dusk Till Dawn (1996)', 3.119565217391304)),
 ('Crime', (17, 'From Dusk Till Dawn (1996)', 3.119565217391304)),
 ('Horror', (17, 'From Dusk Till Dawn (1996)', 3.119565217391304)),
 ('Thriller', (17, 'From Dusk Till Dawn (1996)', 3.119565217391304)),
 ('Action', (21, 'Muppet Treasure Island (1996)', 2.761904761904762)),
 ('Adventure', (21, 'Muppet Treasure Island (1996)', 2.761904761904762)),
 ('Comedy', (21, 'Muppet Treasure Island (1996)', 2.761904761904762)),
 ('Musical', (21, 'Muppet Treasure Island (1996)', 2.761904761904762)),
 ('Thriller', (21, 'Muppet Treasure Island (1996)', 2.761904761904762)),
 ('Action', (22, 'Braveheart (1995)', 4.151515151515151)),
 ('Drama', (22, 'Braveheart (1995)', 4.151515151515151))

In [22]:
#Step 7: Display the top 3 movies for each genre

#group the movies by genre 
result = result.groupByKey().flatMap(lambda x: [(x[0], data) for data in x[1]])
#map the corresponding values of each genre and sort based on ratings in descending order
group_genre = result.groupByKey().mapValues(lambda values: sorted(values, key=lambda x: x[2], reverse=True))
# Take the top three movies in each genre and sort by genre name
top_three = group_genre.mapValues(lambda x: x[:3]).sortByKey()
top_three.collect()

[('Action',
  [(22, 'Braveheart (1995)', 4.151515151515151),
   (117, 'Rock, The (1996)', 3.693121693121693),
   (222, 'Star Trek: First Contact (1996)', 3.66027397260274)]),
 ('Adventure',
  [(500, 'Fly Away Home (1996)', 3.903225806451613),
   (117, 'Rock, The (1996)', 3.693121693121693),
   (222, 'Star Trek: First Contact (1996)', 3.66027397260274)]),
 ('Animation',
  [(408, 'Close Shave, A (1995)', 4.491071428571429),
   (114,
    'Wallace & Gromit: The Best of Aardman Animation (1996)',
    4.447761194029851),
   (1240, 'Ghost in the Shell (Kokaku kidotai) (1995)', 3.6538461538461537)]),
 ("Children's",
  [(500, 'Fly Away Home (1996)', 3.903225806451613),
   (304, 'Fly Away Home (1996)', 3.5369127516778525),
   (596, 'Hunchback of Notre Dame, The (1996)', 3.377952755905512)]),
 ('Comedy',
  [(1500, 'Santa with Muscles (1996)', 5.0),
   (408, 'Close Shave, A (1995)', 4.491071428571429),
   (1080, 'Celestial Clockwork (1994)', 4.0)]),
 ('Crime',
  [(1505, 'Killer: A Journal of Murde

In [25]:
#Step 8: Save each set of genre and top 3 movies to a RDD file
for genre_desc in genre_description:
    for genre,movie_info in top_three.collect():
        if genre_desc == genre:
            filename = f"{genre_desc}.txt"
            rdd = sc.parallelize([(genre),(movie_info)]).coalesce(1).saveAsTextFile(filename)

## 4c)

In [23]:
#Create an age-group category of: [0,6], (6,12], (12,18], (18,30], (30,50], 50+ 
#and assign corresponding category to user/reviewer based on his/her age. 

#load the mov_user text file
mov_user = sc.textFile("mov_user.dat")

#create an if-else statement to match the age with the corresponding age-group
age_category = mov_user.map(lambda line: line.split('|')) \
                  .map(lambda x: (x[0], int(x[1]), '0-6' if 0 <= int(x[1]) <= 6
                                                    else '6-12' if 6 < int(x[1]) <= 12
                                                    else '12-18' if 12 < int(x[1]) <= 18
                                                    else '18-30' if 18 < int(x[1]) <= 30
                                                    else '30-50' if 30 < int(x[1]) <= 50
                                                    else '50+', 
                                  x[2], x[3], x[4]))
#display the output
age_category.collect()

[('1', 24, '18-30', 'M', 'technician', '85711'),
 ('2', 53, '50+', 'F', 'other', '94043'),
 ('3', 23, '18-30', 'M', 'writer', '32067'),
 ('4', 24, '18-30', 'M', 'technician', '43537'),
 ('5', 33, '30-50', 'F', 'other', '15213'),
 ('6', 42, '30-50', 'M', 'executive', '98101'),
 ('7', 57, '50+', 'M', 'administrator', '91344'),
 ('8', 36, '30-50', 'M', 'administrator', '05201'),
 ('9', 29, '18-30', 'M', 'student', '01002'),
 ('10', 53, '50+', 'M', 'lawyer', '90703'),
 ('11', 39, '30-50', 'F', 'other', '30329'),
 ('12', 28, '18-30', 'F', 'other', '06405'),
 ('13', 47, '30-50', 'M', 'educator', '29206'),
 ('14', 45, '30-50', 'M', 'scientist', '55106'),
 ('15', 49, '30-50', 'F', 'educator', '97301'),
 ('16', 21, '18-30', 'M', 'entertainment', '10309'),
 ('17', 30, '18-30', 'M', 'programmer', '06355'),
 ('18', 35, '30-50', 'F', 'other', '37212'),
 ('19', 40, '30-50', 'M', 'librarian', '02138'),
 ('20', 42, '30-50', 'F', 'homemaker', '95660'),
 ('21', 26, '18-30', 'M', 'writer', '30068'),
 ('2

In [24]:
#get the movie id and movie name from the mov_item file

#load the mov_item file and extract out only the first 2 columns which are the movie_id and movie_name
movies_agegroup = sc.textFile("mov_item.dat").map(lambda line: (line.split('|')[0], line.split('|')[1]))
#display the output
movies_agegroup.collect()

[('1', 'Toy Story (1995)'),
 ('2', 'GoldenEye (1995)'),
 ('3', 'Four Rooms (1995)'),
 ('4', 'Get Shorty (1995)'),
 ('5', 'Copycat (1995)'),
 ('6', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 ('7', 'Twelve Monkeys (1995)'),
 ('8', 'Babe (1995)'),
 ('9', 'Dead Man Walking (1995)'),
 ('10', 'Richard III (1995)'),
 ('11', 'Seven (Se7en) (1995)'),
 ('12', 'Usual Suspects, The (1995)'),
 ('13', 'Mighty Aphrodite (1995)'),
 ('14', 'Postino, Il (1994)'),
 ('15', "Mr. Holland's Opus (1995)"),
 ('16', 'French Twist (Gazon maudit) (1995)'),
 ('17', 'From Dusk Till Dawn (1996)'),
 ('18', 'White Balloon, The (1995)'),
 ('19', "Antonia's Line (1995)"),
 ('20', 'Angels and Insects (1995)'),
 ('21', 'Muppet Treasure Island (1996)'),
 ('22', 'Braveheart (1995)'),
 ('23', 'Taxi Driver (1976)'),
 ('24', 'Rumble in the Bronx (1995)'),
 ('25', 'Birdcage, The (1996)'),
 ('26', 'Brothers McMullen, The (1995)'),
 ('27', 'Bad Boys (1995)'),
 ('28', 'Apollo 13 (1995)'),
 ('29', 'Batman Forever (19

In [25]:
#get the user id and agegroup from the age_category

#extract out the user_id and age-group and put in a new rdd
age_id = age_category.map(lambda line: (line[0], line[2]))
#display the output
age_id.collect()

[('1', '18-30'),
 ('2', '50+'),
 ('3', '18-30'),
 ('4', '18-30'),
 ('5', '30-50'),
 ('6', '30-50'),
 ('7', '50+'),
 ('8', '30-50'),
 ('9', '18-30'),
 ('10', '50+'),
 ('11', '30-50'),
 ('12', '18-30'),
 ('13', '30-50'),
 ('14', '30-50'),
 ('15', '30-50'),
 ('16', '18-30'),
 ('17', '18-30'),
 ('18', '30-50'),
 ('19', '30-50'),
 ('20', '30-50'),
 ('21', '18-30'),
 ('22', '18-30'),
 ('23', '18-30'),
 ('24', '18-30'),
 ('25', '30-50'),
 ('26', '30-50'),
 ('27', '30-50'),
 ('28', '30-50'),
 ('29', '30-50'),
 ('30', '6-12'),
 ('31', '18-30'),
 ('32', '18-30'),
 ('33', '18-30'),
 ('34', '30-50'),
 ('35', '18-30'),
 ('36', '18-30'),
 ('37', '18-30'),
 ('38', '18-30'),
 ('39', '30-50'),
 ('40', '30-50'),
 ('41', '30-50'),
 ('42', '18-30'),
 ('43', '18-30'),
 ('44', '18-30'),
 ('45', '18-30'),
 ('46', '18-30'),
 ('47', '50+'),
 ('48', '30-50'),
 ('49', '18-30'),
 ('50', '18-30'),
 ('51', '18-30'),
 ('52', '12-18'),
 ('53', '18-30'),
 ('54', '18-30'),
 ('55', '30-50'),
 ('56', '18-30'),
 ('57', '1

In [26]:
#get the userid, movieid, rating from mov_rating.file

#extract out userid, movieid, rating columns into a new rdd
age_ratings = sc.textFile("mov_rating.dat").map(lambda line: line.split('\t')).map(lambda x: (x[0], x[1], x[2]))
age_ratings.collect()

[('196', '242', '3'),
 ('186', '302', '3'),
 ('22', '377', '1'),
 ('244', '51', '2'),
 ('166', '346', '1'),
 ('298', '474', '4'),
 ('115', '265', '2'),
 ('253', '465', '5'),
 ('305', '451', '3'),
 ('6', '86', '3'),
 ('62', '257', '2'),
 ('286', '1014', '5'),
 ('200', '222', '5'),
 ('210', '40', '3'),
 ('224', '29', '3'),
 ('303', '785', '3'),
 ('122', '387', '5'),
 ('194', '274', '2'),
 ('291', '1042', '4'),
 ('234', '1184', '2'),
 ('119', '392', '4'),
 ('167', '486', '4'),
 ('299', '144', '4'),
 ('291', '118', '2'),
 ('308', '1', '4'),
 ('95', '546', '2'),
 ('38', '95', '5'),
 ('102', '768', '2'),
 ('63', '277', '4'),
 ('160', '234', '5'),
 ('50', '246', '3'),
 ('301', '98', '4'),
 ('225', '193', '4'),
 ('290', '88', '4'),
 ('97', '194', '3'),
 ('157', '274', '4'),
 ('181', '1081', '1'),
 ('278', '603', '5'),
 ('276', '796', '1'),
 ('7', '32', '4'),
 ('10', '16', '4'),
 ('284', '304', '4'),
 ('201', '979', '2'),
 ('276', '564', '3'),
 ('287', '327', '5'),
 ('246', '201', '5'),
 ('242'

In [27]:
#Join the 3 Rdds to get a complete rdd with all of the information

#first join age-group with ratings rdd
agegroup_ratings = age_id.map(lambda x: (x[0], x[1])) \
    .join(age_ratings.map(lambda x: (str(x[0]), (x[1], x[2]))))

#next join the agegroup_ratings with movies_age-group
final_rdd = agegroup_ratings.map(lambda x: (x[1][1][0], (x[0], x[1][0], x[1][1][1]))) \
    .join(movies_agegroup.map(lambda x: (x[0], x[1])))

final_rdd.collect()

[('210', (('4', '18-30', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('5', '30-50', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('22', '18-30', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('43', '18-30', '5'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('51', '18-30', '4'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('64', '30-50', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('77', '18-30', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('102', '30-50', '3'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('119', '30-50', '5'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('130', '18-30', '5'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('151', '30-50', '4'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('187', '18-30', '4'), 'Indiana Jones and the Last Crusade (1989)')),
 ('210', (('188', '30-50', '4'), 'Indiana Jones a

In [28]:
#get the top 30 movies for each age-group

#rearrange the rdd above to the following format (userid, age-group, movie id, movie name, average rating)
all_together = final_rdd.map(lambda x: (x[1][0][0], x[1][0][1], x[0], x[1][1], int(x[1][0][2])))

#from the all_together rdd remove the userid column
#get all the ratings for each movie with the same userid and agegroup
#present the data in age-group, movie id, movie name, average rating
#groupby age-group and sort by ratings in descending order
#take top 30
display_rdd = all_together.map(lambda x: ((x[1], x[2], x[3]), [x[4]])) \
                .reduceByKey(lambda x, y: x + y) \
                .map(lambda x: (x[0][0], x[0][1], x[0][2], sum(x[1])/len(x[1]))) \
                .groupBy(lambda x: (x[0])).mapValues(lambda values: sorted(values, key=lambda x: x[3], reverse=True)) \
                .mapValues(lambda x: x[:30])

#display the output
display_rdd.collect()

[('30-50',
  [('30-50', '1450', 'Golden Earrings (1947)', 5.0),
   ('30-50', '851', 'Two or Three Things I Know About Her (1966)', 5.0),
   ('30-50', '1536', 'Aiqing wansui (1994)', 5.0),
   ('30-50', '1389', 'Mondo (1996)', 5.0),
   ('30-50', '1398', 'Anna (1996)', 5.0),
   ('30-50', '1472', 'Visitors, The (Visiteurs, Les) (1993)', 5.0),
   ('30-50', '1306', 'Delta of Venus (1994)', 5.0),
   ('30-50', '1122', 'They Made Me a Criminal (1939)', 5.0),
   ('30-50', '1358', 'The Deadly Cure (1996)', 5.0),
   ('30-50', '1558', 'Aparajito (1956)', 5.0),
   ('30-50', '1367', 'Faust (1994)', 5.0),
   ('30-50', '1347', 'Ballad of Narayama, The (Narayama Bushiko) (1958)', 5.0),
   ('30-50', '1653', 'Entertaining Angels: The Dorothy Day Story (1996)', 5.0),
   ('30-50', '1357', 'For the Moment (1994)', 5.0),
   ('30-50', '1491', 'Tough and Deadly (1995)', 5.0),
   ('30-50', '814', 'Great Day in Harlem, A (1994)', 5.0),
   ('30-50', '1429', 'Sliding Doors (1998)', 5.0),
   ('30-50', '1597', 'Rompe

In [32]:
#Save each set of age-group and top 30 movies to a RDD file
for age,movie_list in display_rdd.collect():
    filename = f"{age}.txt"
    rdd = sc.parallelize([(age),(movie_list)]).coalesce(1).saveAsTextFile(filename)

## 4d)

In [29]:
#extract lines from mov_item file where the month of release date is in may,june,july
#since the mov_item rdd is already loaded in 4a) bullet point 5
summer_movies = mov_item.filter(lambda line: line.split('|')[2][3:6] in ['May', 'Jun', 'Jul'])
summer_movies.collect()

['93|Welcome to the Dollhouse (1995)|24-May-1996||http://us.imdb.com/Title?Welcome+to+the+Dollhouse+(1995)|0|0|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '107|Moll Flanders (1996)|14-Jun-1996||http://us.imdb.com/M/title-exact?Moll%20Flanders%20(1996)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '112|Flipper (1996)|10-May-1996||http://us.imdb.com/M/title-exact?Flipper%20(1996)|0|0|1|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '117|Rock, The (1996)|07-Jun-1996||http://us.imdb.com/M/title-exact?Rock,%20The%20(1996)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '118|Twister (1996)|10-May-1996||http://us.imdb.com/M/title-exact?Twister%20(1996)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '120|Striptease (1996)|28-Jun-1996||http://us.imdb.com/M/title-exact?Striptease%20(1996)|0|0|0|0|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0',
 '121|Independence Day (ID4) (1996)|03-Jul-1996||http://us.imdb.com/M/title-exact?Independence%20Day%20(1996)|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|1|0',
 '122|Cable Guy, The (1996)|14-Jun-1996||http://us.imdb.c

In [30]:
#Map the genres to each of the movie

#from the rdd above, we only need the Movie ID, Movie name and the genre.
#Filter out the valid fields and place the binary values of genre in a list for easier viewing
formatted_summermovies = summer_movies.map(lambda line: line.split('|')).map(lambda columns: (int(columns[0]), columns[1], [int(x) for x in columns[5:]]))


#use the genre_description and function from 4b
def genre_mapping(genre):
    mapped_genres = [genre_description[i] for i, value in enumerate(genre) if value == 1]
    return mapped_genres

#The function is then used to map each genre in the list for each movie
summer_mapped_movies = formatted_summermovies.map(lambda x: (x[0], x[1], genre_mapping(x[2])))

summer_mapped_movies.collect()

[(93, 'Welcome to the Dollhouse (1995)', ['Comedy', 'Drama']),
 (107, 'Moll Flanders (1996)', ['Drama']),
 (112, 'Flipper (1996)', ['Adventure', "Children's"]),
 (117, 'Rock, The (1996)', ['Action', 'Adventure', 'Thriller']),
 (118, 'Twister (1996)', ['Action', 'Adventure', 'Thriller']),
 (120, 'Striptease (1996)', ['Comedy', 'Crime']),
 (121, 'Independence Day (ID4) (1996)', ['Action', 'Sci-Fi', 'War']),
 (122, 'Cable Guy, The (1996)', ['Comedy']),
 (123, 'Frighteners, The (1996)', ['Comedy', 'Horror']),
 (124, 'Lone Star (1996)', ['Drama', 'Mystery']),
 (125, 'Phenomenon (1996)', ['Drama', 'Romance']),
 (128, 'Supercop (1992)', ['Action', 'Thriller']),
 (249, 'Austin Powers: International Man of Mystery (1997)', ['Comedy']),
 (250, 'Fifth Element, The (1997)', ['Action', 'Sci-Fi']),
 (251, 'Shall We Dance? (1996)', ['Comedy']),
 (252,
  'Lost World: Jurassic Park, The (1997)',
  ['Action', 'Adventure', 'Sci-Fi', 'Thriller']),
 (253, 'Pillow Book, The (1995)', ['Drama', 'Romance']),
 

In [31]:
#Get the respective ratings for each movie in the Rdd above

#get the list of movie_ids
summer_releases = summer_mapped_movies.map(lambda x: x[0]).collect()
#load the mov_rating file and view lines with the movie_id collected from titles_released
summer_mov_rating = sc.textFile("mov_rating.dat").map(lambda line: line.split('\t')).map(lambda columns: (int(columns[0]), int(columns[1]), int(columns[2]), columns[3])).filter(lambda x: (x[1] in summer_releases))
#for each movie_id, get the respective list of ratings
summer_mov_rating = summer_mov_rating.map(lambda x: (x[1], x[2])).groupByKey().mapValues(list)
#get the movie_id, number of ratings for each movie and average rating for each movie and set movie_id as key
summer_filtered_ratings = summer_mov_rating.map(lambda line: (line[0], len(line[1]), sum(int(rating) for rating in line[1]) / len(line[1]))).map(lambda x: (x[0], (x[1], x[2])))
summer_filtered_ratings.collect()

[(118, (293, 3.2150170648464163)),
 (1016, (137, 3.45985401459854)),
 (1336, (10, 1.8)),
 (298, (194, 3.768041237113402)),
 (258, (509, 3.8035363457760316)),
 (930, (80, 2.7)),
 (256, (16, 3.5625)),
 (410, (162, 3.1296296296296298)),
 (250, (197, 3.4771573604060912)),
 (596, (127, 3.377952755905512)),
 (1060, (39, 3.051282051282051)),
 (252, (158, 2.9430379746835444)),
 (922, (34, 3.823529411764706)),
 (824, (49, 2.7551020408163267)),
 (1198, (7, 3.142857142857143)),
 (128, (65, 3.5384615384615383)),
 (120, (67, 2.2388059701492535)),
 (122, (106, 2.339622641509434)),
 (472, (158, 3.0822784810126582)),
 (1150, (9, 3.111111111111111)),
 (1334, (2, 1.0)),
 (1160, (26, 3.6153846153846154)),
 (282, (232, 3.685344827586207)),
 (826, (80, 2.575)),
 (254, (62, 2.4516129032258065)),
 (1342, (2, 2.5)),
 (536, (10, 3.3)),
 (1320, (1, 1.0)),
 (124, (187, 4.053475935828877)),
 (1262, (8, 3.625)),
 (1328, (6, 3.1666666666666665)),
 (830, (1, 1.0)),
 (978, (27, 2.5185185185185186)),
 (1292, (3, 1.666

In [32]:
#Rearrange the mapped_movies RDD to get the genre of each movie as the key
summer_movie_pairs = summer_mapped_movies.flatMap(lambda x: [(genre, (x[0], x[1])) for genre in x[2]])
summer_movie_pairs.collect()

[('Comedy', (93, 'Welcome to the Dollhouse (1995)')),
 ('Drama', (93, 'Welcome to the Dollhouse (1995)')),
 ('Drama', (107, 'Moll Flanders (1996)')),
 ('Adventure', (112, 'Flipper (1996)')),
 ("Children's", (112, 'Flipper (1996)')),
 ('Action', (117, 'Rock, The (1996)')),
 ('Adventure', (117, 'Rock, The (1996)')),
 ('Thriller', (117, 'Rock, The (1996)')),
 ('Action', (118, 'Twister (1996)')),
 ('Adventure', (118, 'Twister (1996)')),
 ('Thriller', (118, 'Twister (1996)')),
 ('Comedy', (120, 'Striptease (1996)')),
 ('Crime', (120, 'Striptease (1996)')),
 ('Action', (121, 'Independence Day (ID4) (1996)')),
 ('Sci-Fi', (121, 'Independence Day (ID4) (1996)')),
 ('War', (121, 'Independence Day (ID4) (1996)')),
 ('Comedy', (122, 'Cable Guy, The (1996)')),
 ('Comedy', (123, 'Frighteners, The (1996)')),
 ('Horror', (123, 'Frighteners, The (1996)')),
 ('Drama', (124, 'Lone Star (1996)')),
 ('Mystery', (124, 'Lone Star (1996)')),
 ('Drama', (125, 'Phenomenon (1996)')),
 ('Romance', (125, 'Phenome

In [33]:
#Create new RDD to join the ratings rdd and the genre_movie_pairs rdd based on movie_id 
#to get an RDD with genre, movie_id, movie_name and corresponding average rating

#Initialize an empty list
summer_result = []
#Create a for loop for both ratings and genre_movie rdd and append 
#genre, movie_id, movie_name and corresponding average rating only if the movie_id between the two RDDs
#are the same
for genre, (movie_id, movie_name) in summer_movie_pairs.collect():
    for movie_id2, (total_reviews, avg_rating) in summer_filtered_ratings.collect():
        if movie_id == movie_id2:
            summer_result.append((genre, (movie_id, movie_name, avg_rating)))
            
#paralleize the list into an RDD and display the result
summer_result = sc.parallelize(summer_result)
summer_result.collect()

[('Comedy', (93, 'Welcome to the Dollhouse (1995)', 3.7232142857142856)),
 ('Drama', (93, 'Welcome to the Dollhouse (1995)', 3.7232142857142856)),
 ('Drama', (107, 'Moll Flanders (1996)', 3.5)),
 ('Adventure', (112, 'Flipper (1996)', 2.45)),
 ("Children's", (112, 'Flipper (1996)', 2.45)),
 ('Action', (117, 'Rock, The (1996)', 3.693121693121693)),
 ('Adventure', (117, 'Rock, The (1996)', 3.693121693121693)),
 ('Thriller', (117, 'Rock, The (1996)', 3.693121693121693)),
 ('Action', (118, 'Twister (1996)', 3.2150170648464163)),
 ('Adventure', (118, 'Twister (1996)', 3.2150170648464163)),
 ('Thriller', (118, 'Twister (1996)', 3.2150170648464163)),
 ('Comedy', (120, 'Striptease (1996)', 2.2388059701492535)),
 ('Crime', (120, 'Striptease (1996)', 2.2388059701492535)),
 ('Action', (121, 'Independence Day (ID4) (1996)', 3.438228438228438)),
 ('Sci-Fi', (121, 'Independence Day (ID4) (1996)', 3.438228438228438)),
 ('War', (121, 'Independence Day (ID4) (1996)', 3.438228438228438)),
 ('Comedy', (12

In [34]:
#Display the top 3 movies for each genre

#group the movies by genre 
summer_result = summer_result.groupByKey().flatMap(lambda x: [(x[0], data) for data in x[1]])
#map the corresponding values of each genre and sort based on ratings in descending order
summer_grouped = summer_result.groupByKey().mapValues(lambda values: sorted(values, key=lambda x: x[2], reverse=True))
# Take the top three movies in each genre and sort by genre name
top_three_summer_movies = summer_grouped.mapValues(lambda x: x[:3]).sortByKey()
top_three_summer_movies.collect()

[('Action',
  [(298, 'Face/Off (1997)', 3.768041237113402),
   (257, 'Men in Black (1997)', 3.745874587458746),
   (117, 'Rock, The (1996)', 3.693121693121693)]),
 ('Adventure',
  [(257, 'Men in Black (1997)', 3.745874587458746),
   (117, 'Rock, The (1996)', 3.693121693121693),
   (993, 'Hercules (1997)', 3.515151515151515)]),
 ('Animation',
  [(993, 'Hercules (1997)', 3.515151515151515),
   (596, 'Hunchback of Notre Dame, The (1996)', 3.377952755905512)]),
 ("Children's",
  [(993, 'Hercules (1997)', 3.515151515151515),
   (596, 'Hunchback of Notre Dame, The (1996)', 3.377952755905512),
   (1060, 'Adventures of Pinocchio, The (1996)', 3.051282051282051)]),
 ('Comedy',
  [(251, 'Shall We Dance? (1996)', 4.260869565217392),
   (1080, 'Celestial Clockwork (1994)', 4.0),
   (936, 'Brassed Off (1996)', 3.9375)]),
 ('Crime',
  [(1226, 'Night Falls on Manhattan (1997)', 3.375),
   (1187, 'Switchblade Sisters (1975)', 3.230769230769231),
   (1198, 'Purple Noon (1960)', 3.142857142857143)]),
 (

In [39]:
#Save each set of genre and top 3 movies to a RDD file
for genre_desc in genre_description:
    for genre,movie_info in top_three_summer_movies.collect():
        if genre_desc == genre:
            filename = f"{genre_desc}_4d.txt"
            rdd = sc.parallelize([(genre),(movie_info)]).coalesce(1).saveAsTextFile(filename)

## 4e)

In [50]:
#load the mov_user file and extract out the user id and occupation
mov_user = sc.textFile("mov_user.dat").map(lambda line: line.split('|')).map(lambda x: (x[0], x[3]))
mov_user.collect()

[('1', 'technician'),
 ('2', 'other'),
 ('3', 'writer'),
 ('4', 'technician'),
 ('5', 'other'),
 ('6', 'executive'),
 ('7', 'administrator'),
 ('8', 'administrator'),
 ('9', 'student'),
 ('10', 'lawyer'),
 ('11', 'other'),
 ('12', 'other'),
 ('13', 'educator'),
 ('14', 'scientist'),
 ('15', 'educator'),
 ('16', 'entertainment'),
 ('17', 'programmer'),
 ('18', 'other'),
 ('19', 'librarian'),
 ('20', 'homemaker'),
 ('21', 'writer'),
 ('22', 'writer'),
 ('23', 'artist'),
 ('24', 'artist'),
 ('25', 'engineer'),
 ('26', 'engineer'),
 ('27', 'librarian'),
 ('28', 'writer'),
 ('29', 'programmer'),
 ('30', 'student'),
 ('31', 'artist'),
 ('32', 'student'),
 ('33', 'student'),
 ('34', 'administrator'),
 ('35', 'homemaker'),
 ('36', 'student'),
 ('37', 'student'),
 ('38', 'other'),
 ('39', 'entertainment'),
 ('40', 'scientist'),
 ('41', 'engineer'),
 ('42', 'administrator'),
 ('43', 'librarian'),
 ('44', 'technician'),
 ('45', 'programmer'),
 ('46', 'marketing'),
 ('47', 'marketing'),
 ('48', 'a

In [51]:
#load the mov_genre file
mov_genre = sc.textFile("mov_genre.dat")
mov_genre.collect()

['unknown|0',
 'Action|1',
 'Adventure|2',
 'Animation|3',
 "Children's|4",
 'Comedy|5',
 'Crime|6',
 'Documentary|7',
 'Drama|8',
 'Fantasy|9',
 'Film-Noir|10',
 'Horror|11',
 'Musical|12',
 'Mystery|13',
 'Romance|14',
 'Sci-Fi|15',
 'Thriller|16',
 'War|17',
 'Western|18']

In [52]:
#load the movie_item file
mov_item = sc.textFile("mov_item.dat")
mov_item.collect()

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0',
 '6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0',
 '8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title

In [53]:
#Map the genres to each of the movie

#from the rdd above, we only need the Movie ID, Movie name and the genre.
#Filter out the valid fields and place the binary values of genre in a list for easier viewing
occupation_genre_movies = mov_item.map(lambda line: line.split('|')).map(lambda columns: (int(columns[0]), columns[1], [int(x) for x in columns[5:]]))


#use the genre_description and function from 4b
def genre_mapping(genre):
    mapped_genres = [genre_description[i] for i, value in enumerate(genre) if value == 1]
    return mapped_genres

#The function is then used to map each genre in the list for each movie
necessary_items_mapped = occupation_genre_movies.map(lambda x: (x[0], x[1], genre_mapping(x[2])))

necessary_items_mapped.collect()

[(1, 'Toy Story (1995)', ['Animation', "Children's", 'Comedy']),
 (2, 'GoldenEye (1995)', ['Action', 'Adventure', 'Thriller']),
 (3, 'Four Rooms (1995)', ['Thriller']),
 (4, 'Get Shorty (1995)', ['Action', 'Comedy', 'Drama']),
 (5, 'Copycat (1995)', ['Crime', 'Drama', 'Thriller']),
 (6, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', ['Drama']),
 (7, 'Twelve Monkeys (1995)', ['Drama', 'Sci-Fi']),
 (8, 'Babe (1995)', ["Children's", 'Comedy', 'Drama']),
 (9, 'Dead Man Walking (1995)', ['Drama']),
 (10, 'Richard III (1995)', ['Drama', 'War']),
 (11, 'Seven (Se7en) (1995)', ['Crime', 'Thriller']),
 (12, 'Usual Suspects, The (1995)', ['Crime', 'Thriller']),
 (13, 'Mighty Aphrodite (1995)', ['Comedy']),
 (14, 'Postino, Il (1994)', ['Drama', 'Romance']),
 (15, "Mr. Holland's Opus (1995)", ['Drama']),
 (16, 'French Twist (Gazon maudit) (1995)', ['Comedy', 'Romance']),
 (17,
  'From Dusk Till Dawn (1996)',
  ['Action', 'Comedy', 'Crime', 'Horror', 'Thriller']),
 (18, 'White Balloon, The

In [54]:
#Rearrange the mapped_movies RDD to get the genre of each movie as the key
open_genre = necessary_items_mapped.flatMap(lambda x: [(genre, (x[0], x[1])) for genre in x[2]])
open_genre.collect()

[('Animation', (1, 'Toy Story (1995)')),
 ("Children's", (1, 'Toy Story (1995)')),
 ('Comedy', (1, 'Toy Story (1995)')),
 ('Action', (2, 'GoldenEye (1995)')),
 ('Adventure', (2, 'GoldenEye (1995)')),
 ('Thriller', (2, 'GoldenEye (1995)')),
 ('Thriller', (3, 'Four Rooms (1995)')),
 ('Action', (4, 'Get Shorty (1995)')),
 ('Comedy', (4, 'Get Shorty (1995)')),
 ('Drama', (4, 'Get Shorty (1995)')),
 ('Crime', (5, 'Copycat (1995)')),
 ('Drama', (5, 'Copycat (1995)')),
 ('Thriller', (5, 'Copycat (1995)')),
 ('Drama', (6, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)')),
 ('Drama', (7, 'Twelve Monkeys (1995)')),
 ('Sci-Fi', (7, 'Twelve Monkeys (1995)')),
 ("Children's", (8, 'Babe (1995)')),
 ('Comedy', (8, 'Babe (1995)')),
 ('Drama', (8, 'Babe (1995)')),
 ('Drama', (9, 'Dead Man Walking (1995)')),
 ('Drama', (10, 'Richard III (1995)')),
 ('War', (10, 'Richard III (1995)')),
 ('Crime', (11, 'Seven (Se7en) (1995)')),
 ('Thriller', (11, 'Seven (Se7en) (1995)')),
 ('Crime', (12, 'Usual Sus

In [55]:
#load the mov_rating file and extract out the userid,movieid,rating
mov_rating = sc.textFile("mov_rating.dat").map(lambda line: line.split('\t')).map(lambda columns: (int(columns[0]), int(columns[1]), int(columns[2])))
mov_rating.collect()

[(196, 242, 3),
 (186, 302, 3),
 (22, 377, 1),
 (244, 51, 2),
 (166, 346, 1),
 (298, 474, 4),
 (115, 265, 2),
 (253, 465, 5),
 (305, 451, 3),
 (6, 86, 3),
 (62, 257, 2),
 (286, 1014, 5),
 (200, 222, 5),
 (210, 40, 3),
 (224, 29, 3),
 (303, 785, 3),
 (122, 387, 5),
 (194, 274, 2),
 (291, 1042, 4),
 (234, 1184, 2),
 (119, 392, 4),
 (167, 486, 4),
 (299, 144, 4),
 (291, 118, 2),
 (308, 1, 4),
 (95, 546, 2),
 (38, 95, 5),
 (102, 768, 2),
 (63, 277, 4),
 (160, 234, 5),
 (50, 246, 3),
 (301, 98, 4),
 (225, 193, 4),
 (290, 88, 4),
 (97, 194, 3),
 (157, 274, 4),
 (181, 1081, 1),
 (278, 603, 5),
 (276, 796, 1),
 (7, 32, 4),
 (10, 16, 4),
 (284, 304, 4),
 (201, 979, 2),
 (276, 564, 3),
 (287, 327, 5),
 (246, 201, 5),
 (242, 1137, 5),
 (249, 241, 5),
 (99, 4, 5),
 (178, 332, 3),
 (251, 100, 4),
 (81, 432, 2),
 (260, 322, 4),
 (25, 181, 5),
 (59, 196, 5),
 (72, 679, 2),
 (87, 384, 4),
 (290, 143, 5),
 (42, 423, 5),
 (292, 515, 4),
 (115, 20, 3),
 (20, 288, 1),
 (201, 219, 4),
 (13, 526, 3),
 (246,

In [56]:
#Join the Rdds to get a complete rdd with all of the information

#firstly, join occupation and rating rdd
occupation_rating = mov_user.map(lambda x: (x[0], x[1])) \
    .join(mov_rating.map(lambda x: (str(x[0]), (x[1], x[2]))))

#map the rdd above with the open_genre rdd
mapped_rdd_final = occupation_rating.map(lambda x: (x[1][1][0], (x[0], x[1][0], x[1][1][1]))) \
    .join(open_genre.map(lambda x: (x[1][0], (x[0], x[1][1]))))

# Show the mapped results
mapped_rdd_final.collect()

[(264, (('4', 'technician', 3), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('4', 'technician', 3), ('Thriller', 'Mimic (1997)'))),
 (264, (('21', 'writer', 3), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('21', 'writer', 3), ('Thriller', 'Mimic (1997)'))),
 (264, (('29', 'programmer', 3), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('29', 'programmer', 3), ('Thriller', 'Mimic (1997)'))),
 (264, (('57', 'none', 2), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('57', 'none', 2), ('Thriller', 'Mimic (1997)'))),
 (264, (('102', 'programmer', 2), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('102', 'programmer', 2), ('Thriller', 'Mimic (1997)'))),
 (264, (('107', 'scientist', 3), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('107', 'scientist', 3), ('Thriller', 'Mimic (1997)'))),
 (264, (('181', 'executive', 2), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('181', 'executive', 2), ('Thriller', 'Mimic (1997)'))),
 (264, (('250', 'executive', 3), ('Sci-Fi', 'Mimic (1997)'))),
 (264, (('250', 'executive', 3), ('Thriller', 'Mimic (1997)')

In [57]:
#rearrange the rdd above to the following format (genre,occupation,movieid,moviename,rating)
mapped_rdd_final = mapped_rdd_final.map(lambda x: (x[1][1][0], x[1][0][1], x[0], x[1][1][1], x[1][0][2]))

#from the mapped_rdd_final rdd 
#get all the ratings for each movie with the same genre and occupation
#present the data in genre, occupation, movie id, movie name, average rating
#groupby occupation and genre and sort by ratings in descending order
#take top 3
result_final = mapped_rdd_final.map(lambda x: ((x[0], x[1], x[2], x[3]), [x[4]])) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: ((x[0][0], x[0][1]), (x[0][2], x[0][3], sum(x[1])/len(x[1])))) \
    .groupByKey().mapValues(lambda values: sorted(values, key=lambda x: x[3], reverse=True)) \
    .sortByKey() \
    .mapValues(lambda x: x[:3])


result_final.collect()

23/11/07 10:00:32 ERROR Executor: Exception in task 1.0 in stage 1732.0 (TID 2011)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harippriyab/anaconda3/lib

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1732.0 failed 1 times, most recent failure: Lost task 1.0 in stage 1732.0 (TID 2011) (harippriyas-mbp executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harippriyab/anaconda3/lib/python3.11/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harippriyab

In [48]:
#stop Spark Session
sc.stop()

## 4f)

In [49]:
#Start Spark RDD Session
import re
import sys
import os
os.environ['PYSPARK_PYTHON'] = sys.executable
import numpy
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkConf, SparkContext


conf = SparkConf()
sc = SparkContext(conf=conf)

In [50]:
#load the mov_rating file
mov_rating = sc.textFile("mov_rating.dat")
mov_rating.collect()

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013',
 '62\t257\t2\t879372434',
 '286\t1014\t5\t879781125',
 '200\t222\t5\t876042340',
 '210\t40\t3\t891035994',
 '224\t29\t3\t888104457',
 '303\t785\t3\t879485318',
 '122\t387\t5\t879270459',
 '194\t274\t2\t879539794',
 '291\t1042\t4\t874834944',
 '234\t1184\t2\t892079237',
 '119\t392\t4\t886176814',
 '167\t486\t4\t892738452',
 '299\t144\t4\t877881320',
 '291\t118\t2\t874833878',
 '308\t1\t4\t887736532',
 '95\t546\t2\t879196566',
 '38\t95\t5\t892430094',
 '102\t768\t2\t883748450',
 '63\t277\t4\t875747401',
 '160\t234\t5\t876861185',
 '50\t246\t3\t877052329',
 '301\t98\t4\t882075827',
 '225\t193\t4\t879539727',
 '290\t88\t4\t880731963',
 '97\t194\t3\t884238860',
 '157\t274\t4\t886890835',
 '181\t1081\t1\t878962623',
 '278\t603\t5\t

In [51]:
#create a new rdd to store the lines that needs to be added
lines_to_add = sc.parallelize([('0\t50\t5\t881250949'), ('0\t172\t5\t881250949'), ('0\t181\t5\t881250949')])
lines_to_add.collect()

['0\t50\t5\t881250949', '0\t172\t5\t881250949', '0\t181\t5\t881250949']

In [52]:
#create a duplicate record of mov_rating to prevent overriding original data file
mov_rating_duplicate = mov_rating
mov_rating_duplicate.collect()

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013',
 '62\t257\t2\t879372434',
 '286\t1014\t5\t879781125',
 '200\t222\t5\t876042340',
 '210\t40\t3\t891035994',
 '224\t29\t3\t888104457',
 '303\t785\t3\t879485318',
 '122\t387\t5\t879270459',
 '194\t274\t2\t879539794',
 '291\t1042\t4\t874834944',
 '234\t1184\t2\t892079237',
 '119\t392\t4\t886176814',
 '167\t486\t4\t892738452',
 '299\t144\t4\t877881320',
 '291\t118\t2\t874833878',
 '308\t1\t4\t887736532',
 '95\t546\t2\t879196566',
 '38\t95\t5\t892430094',
 '102\t768\t2\t883748450',
 '63\t277\t4\t875747401',
 '160\t234\t5\t876861185',
 '50\t246\t3\t877052329',
 '301\t98\t4\t882075827',
 '225\t193\t4\t879539727',
 '290\t88\t4\t880731963',
 '97\t194\t3\t884238860',
 '157\t274\t4\t886890835',
 '181\t1081\t1\t878962623',
 '278\t603\t5\t

In [53]:
#join the line_to_add rdd with this rdd to insert the data
combined_rdd = lines_to_add.union(mov_rating_duplicate)
combined_rdd.collect()

['0\t50\t5\t881250949',
 '0\t172\t5\t881250949',
 '0\t181\t5\t881250949',
 '196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013',
 '62\t257\t2\t879372434',
 '286\t1014\t5\t879781125',
 '200\t222\t5\t876042340',
 '210\t40\t3\t891035994',
 '224\t29\t3\t888104457',
 '303\t785\t3\t879485318',
 '122\t387\t5\t879270459',
 '194\t274\t2\t879539794',
 '291\t1042\t4\t874834944',
 '234\t1184\t2\t892079237',
 '119\t392\t4\t886176814',
 '167\t486\t4\t892738452',
 '299\t144\t4\t877881320',
 '291\t118\t2\t874833878',
 '308\t1\t4\t887736532',
 '95\t546\t2\t879196566',
 '38\t95\t5\t892430094',
 '102\t768\t2\t883748450',
 '63\t277\t4\t875747401',
 '160\t234\t5\t876861185',
 '50\t246\t3\t877052329',
 '301\t98\t4\t882075827',
 '225\t193\t4\t879539727',
 '290\t88\t4\t880731963',
 '97\t194\t3\t88423886

In [54]:
#reduce the number of partitions to 1 and save it into a textfile
combined_rdd = combined_rdd.coalesce(1)
combined_rdd.saveAsTextFile("mov_rating_final.txt")

In [55]:
#read the new text file and load the rdd
#the last column is not needed as its just the timestamp which is irrelevant here.
new_mov_rating = sc.textFile("mov_rating_final.txt/part-00000")
new_mov_rating.collect()

['0\t50\t5\t881250949',
 '0\t172\t5\t881250949',
 '0\t181\t5\t881250949',
 '196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013',
 '62\t257\t2\t879372434',
 '286\t1014\t5\t879781125',
 '200\t222\t5\t876042340',
 '210\t40\t3\t891035994',
 '224\t29\t3\t888104457',
 '303\t785\t3\t879485318',
 '122\t387\t5\t879270459',
 '194\t274\t2\t879539794',
 '291\t1042\t4\t874834944',
 '234\t1184\t2\t892079237',
 '119\t392\t4\t886176814',
 '167\t486\t4\t892738452',
 '299\t144\t4\t877881320',
 '291\t118\t2\t874833878',
 '308\t1\t4\t887736532',
 '95\t546\t2\t879196566',
 '38\t95\t5\t892430094',
 '102\t768\t2\t883748450',
 '63\t277\t4\t875747401',
 '160\t234\t5\t876861185',
 '50\t246\t3\t877052329',
 '301\t98\t4\t882075827',
 '225\t193\t4\t879539727',
 '290\t88\t4\t880731963',
 '97\t194\t3\t88423886

In [56]:
data = new_mov_rating.map(lambda x: x.split('\t')).map(lambda x:(int(x[0]), int(x[1]), int(x[2])))
ratings = data.map(lambda l: Rating(int(l[0]), int(l[1]), int(l[2])))
 
rank = 20 
numIterations = 15 
 
# #Create the model on the data
model = ALS.train(ratings, rank, numIterations)

# #get top 10 movie recommendations for user id 0
user_id = 0
num_recommendations = 10
user_recommendations = model.recommendProducts(user_id, num_recommendations)
user_recommendations[0:10]

23/11/06 23:56:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/06 23:56:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


[Rating(user=0, product=50, rating=5.026759134271771),
 Rating(user=0, product=181, rating=4.982166961657972),
 Rating(user=0, product=172, rating=4.978533845571766),
 Rating(user=0, product=174, rating=4.770601568306814),
 Rating(user=0, product=313, rating=4.4574352715656875),
 Rating(user=0, product=963, rating=4.417497008336117),
 Rating(user=0, product=210, rating=4.410977884166876),
 Rating(user=0, product=1218, rating=4.381948901032551),
 Rating(user=0, product=96, rating=4.380692135105885),
 Rating(user=0, product=204, rating=4.37021012093426)]

In [57]:
#stop Spark Session
sc.stop()